In [1]:
"""
------------------ import & initial Setting -------------------
"""
# 주피터 환경 설정
%gui qt6

# 표준 라이브러리 및 외부 라이브러리
import sys
import time
import json
import random
import cv2
import numpy as np
from collections import deque
from ultralytics import YOLO

# PyQt6 핵심 설정 및 데이터 타입
from PyQt6.QtCore import (
    Qt, QObject, QThread, QTimer, QCoreApplication, pyqtSignal,
    QRect, QEasingCurve, QPropertyAnimation, QParallelAnimationGroup
)

# PyQt6 GUI 그래픽 및 이미지 처리
from PyQt6.QtGui import (
    QImage, QPixmap, QColor
)

# PyQt6 UI 위젯 요소
from PyQt6.QtWidgets import (
    QApplication, QWidget, QDialog, QLabel, QPushButton,
    QFrame, QVBoxLayout, QStackedWidget, QGraphicsOpacityEffect, QGraphicsDropShadowEffect
)

# PyQt 앱을 실행하기 위한 QApplication 객체 생성 쥬피터 노트에서는 반드시 필요!
app = QApplication.instance()
if not app:
    app = QApplication(sys.argv)  # 이미 존재하면 재사용, 없으면 새로 만듦
"""
---------------- import & initial Setting end -----------------
"""


"""
------------------ File path Setting -------------------
"""
# my file path
# my_model_path = "yolov8m.onnx"
# stop_target_list = ["cup", "bottle"]
# signal_target_list = ["cell phone", "remote"]
my_model_path = "best.onnx"
stop_target_list = ["person"]
signal_target_list = ["circle", "cross"]

my_map_path = "map_data.json"
"""
---------------- File path Setting end -----------------
"""



"""
------------------ Robot SW Class -------------------
"""
class LegosRobot(QObject):
    # [상태 상수 정의 - 하드웨어 프로토콜]
    STOP = -1
    PAUSE = 0
    STRAIGHT = 1
    TURN_RIGHT = 2
    TURN_LEFT = 3
    ADJUST = 7
    TURN_180 = 4

    # 2. 통신용 시그널 정의
    step_signal = pyqtSignal(int, int)   # 현재 위치 (row, col) 전달
    status_signal = pyqtSignal(str)      # 상태 메시지 전달
    mission_finished = pyqtSignal(bool)  # 주행 종료 알림
    image_signal = pyqtSignal(QImage) # 프레임 전송용 시그널 추가
    request_drive = pyqtSignal(tuple, tuple, int) # 시작, 목적지, 모드 전달용

    def __init__(self, model_path):
        super().__init__()

        # 1. 모델 로드
        self.model = YOLO(model_path)

        # 2. 지도 데이터 로드 및 초기화
        self.my_map_list = None
        self.my_map_numpy = None

        # [상태 제어 멤버 변수 초기화]
        self.drive_path = None
        self.all_commands = []
        self.current_cmd_idx = 0
        self.is_completed = False
        self.target_ready_for_next = True   # Target을 지났는지 확인
        self.last_execution_time = 0        # 마지막으로 Target 인식이 실행된 시점
        self.cooldown_duration = 3.0        # Target 인식 최소 간격 (3초)
        self.current_state = self.STOP      # Robot 의 동작 상태 변수
        self.is_action_finished = True
        self.state_before_stop = self.STRAIGHT # 정지 전 상태 저장용
        self.request_drive.connect(self.run_auto_drive)
        self.is_emergency_stop = False # 비상정지 플래그 추가
        self.after_redetected=False

    def stop_immediately(self):
        """외부에서 호출할 비상정지 함수"""
        self.is_emergency_stop = True
        self.move(self.STOP)
        print("🚨 [EMERGENCY] 비상정지 명령 수신!")

    def move(self, new_state):
        # 상태가 변할 때만 로그를 출력하도록 하여 가독성을 높임
        if self.current_state != new_state:
            self.current_state = new_state
            # 새로운 명령이 들어오면 일단 동작 중(False)으로 설정
            self.is_action_finished = False
            if not self.is_emergency_stop:
                msgs = {
                    self.STRAIGHT: "전진 중...",
                    self.TURN_RIGHT: "우회전 중...",
                    self.TURN_LEFT: "좌회전 중...",
                    self.ADJUST: "위치 조정 중...",
                    self.TURN_180: "유턴 중..."
                }
                display_msg = msgs.get(new_state, "대기 중")
                self.status_signal.emit(display_msg)

            if self.current_state == self.STRAIGHT:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (전진 ↑)")

            elif self.current_state == self.TURN_RIGHT:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (우회전 →)")
                time.sleep(1.0)
                self.is_action_finished = True

            elif self.current_state == self.TURN_LEFT:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (좌회전 ←)")
                time.sleep(1.0)
                self.is_action_finished = True

            elif self.current_state == self.PAUSE:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (일시정지)")
                self.is_action_finished = False

            elif self.current_state == self.STOP:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (정지)")
                self.is_action_finished = True

            elif self.current_state == self.ADJUST:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (위치조정)")
                time.sleep(0.5)
                self.current_state = self.STOP # 위치 조정 후 정지
                self.is_action_finished = True

            elif self.current_state == self.TURN_180:
                print(f">>> [MOVE] 신호 발생: {self.current_state} (180도 유턴 ↻)")
                # 90도 회전(1.0초)의 두 배인 2.0초 동안 회전 (로봇 성능에 맞춰 조절)
                time.sleep(2.0)
                self.current_state = self.STOP
                self.is_action_finished = True

    def load_map(self, map_path):
        try:
            with open(map_path, 'r', encoding='utf-8') as f:
                self.my_map_list = json.load(f)
            # 리스트를 NumPy 배열로 변환하여 저장
            self.my_map_numpy = np.array(self.my_map_list, dtype=object)
            print(f">>> [Legos] Map Loaded: {map_path}")
            return True
        except Exception as e:
            print(f">>> [Error] Map Load Failed: {e}")
            return False

    def find_path(self, start, goal):
        # 클래스 멤버인 self.grid_map을 사용 (기존 grid 인자 대체)
        grid = self.my_map_numpy

        # 행과 열의 수를 정의 (NumPy Shape 활용)
        rows, cols = grid.shape

        # 방문 기록을 위한 NumPy 배열 (True | False)
        visited = np.zeros((rows, cols), dtype=bool)
        # 경로 역추적을 위한 부모 좌표 저장 배열 (행, 열 저장용 3차원 배열)
        # 첫 번째 층 : 행 정보 / 두 번째 층 : 열 정보
        parent_map = np.full((rows, cols, 2), -1, dtype=int)

        # 3. 가용 경로 마스크 생성 (NumPy의 강력한 비교 연산)
        # 0이거나 "+"인 곳을 True로 표시
        is_passable = (grid == "R") | (grid == "C")

        # 상, 하, 좌, 우
        # 각 값을 현재 위치 index 에 더하면 각 방향으로 이동한 좌표가 된다.
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        # queue를 deque로 정의 ( 양방향 input이 가능함 | FIFO/LIFO 형식 모두 구현 가능 )
        queue = deque([start])

        # start 좌표에 방문 기록 update
        visited[start] = True

        while queue:
            # queue에 담긴 위치를 하나 pop
            curr_r, curr_c = queue.popleft()

            # 만약 pop 된 위치가 목적지일 경우
            if (curr_r, curr_c) == goal:
                path = []
                # 현재 위치부터 부모 위치를 역탐색하여 path list에 append
                while curr_r != -1:
                    path.append((curr_r, curr_c))
                    curr_r, curr_c = parent_map[curr_r, curr_c]
                # path list의 순서를 역방향으로 뒤집어서 return
                return path[::-1]

            # 아직 목적지가 아닐 경우, 현재 위치에서 이동 가능한 위치들을 방문.
            for dr, dc in directions:
                nr, nc = curr_r + dr, curr_c + dc

                # 경계 검사 및 방문 여부, 통과 가능 여부 확인
                if 0 <= nr < rows and 0 <= nc < cols:
                    if not visited[nr, nc] and is_passable[nr, nc]:
                        visited[nr, nc] = True
                        parent_map[nr, nc] = [curr_r, curr_c]
                        queue.append((nr, nc))

        return None

    def get_commands(self, path):
        # 입력된 경로를 NumPy 배열로 변환
        path = np.array(path)

        if len(path) < 2:
            return ["경로가 너무 짧아 명령을 생성할 수 없습니다."]

        # 1. 모든 이동 벡터를 한 번에 계산 (다음 좌표 - 현재 좌표)
        diffs = np.diff(path, axis=0)

        # 2. 방향 매핑
        heading_lookup = {(-1, 0): 0, (0, 1): 1, (1, 0): 2, (0, -1): 3}
        heading_names = {0: "상(↑)", 1: "우(→)", 2: "하(↓)", 3: "좌(←)"}

        # 모든 구간의 절대 방향(Heading)을 리스트로 변환
        headings = [heading_lookup[tuple(d)] for d in diffs]
        headings = np.array(headings)

        commands = []

        # --- 1단계: 초기 헤딩 설정 ---
        current_heading = headings[0]
        commands.append(f"1단계: 직진 (Go Straight)")

        # --- 2단계: 상대적 회전 계산 (NumPy 차분 활용) ---
        rotations = np.diff(headings) % 4

        # 회전 명령 매핑
        cmd_map = {0: "직진 (Go Straight)", 1: "우회전 (Turn Right)",
                   2: "유턴 (Turn Back)", 3: "좌회전 (Turn Left)"}

        # 계산된 회전값(rotations)을 바탕으로 명령어 생성
        for i, rot in enumerate(rotations):
            target_heading = headings[i+1]
            cmd = cmd_map[rot]
            commands.append(f"{i+2}단계: {cmd}")

        return commands

    def run_auto_drive(self, start_pos, goal_pos, m=0):
            print("run_auto_drive")
            # [경로 탐색 및 이동 경로 정의]
            """
            m=0: 새 경로 탐색 (Normal)
            m=1: 복귀모드 (Return)
            """
            self.is_emergency_stop = False # 주행 시작 전 초기화
            if m == 0:
                self.drive_path = self.find_path(start_pos, goal_pos)
                if self.drive_path:
                    self.all_commands = self.get_commands(self.drive_path)
                else:
                    self.all_commands = ["경로를 찾을 수 없습니다."]
                    return False
            elif m == 1:
                # 복귀 mode
                if self.drive_path:
                    # 기존경로 그대로 뒤집기
                    reversed_path = self.drive_path[::-1]
                    # 뒤집힌 경로 주행 command 재설정 (회전 방향 자동 보정)
                    self.all_commands = self.get_commands(reversed_path)
                    self.drive_path = reversed_path
                    self.move(self.TURN_180)
                else:
                    print(">>> [오류] 이전에 주행한 경로 데이터가 없습니다.")
                    return False

            total_cmds = len(self.all_commands)
            self.current_cmd_idx = 0
            self.is_completed = False
            self.target_ready_for_next = True
            self.last_execution_time = 0

            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("웹캡을 열 수 없습니다.")
                return

            while True:
                if self.is_emergency_stop:
                    print("🚨 [STOP] 비상정지로 인해 루프를 중단합니다.")
                    self.mission_finished.emit(False)
                    return False
                    break
                # [영상 정보 정제]
                ret, frame = cap.read()
                if not ret:
                    print("프레임을 가져올 수 없습니다.")
                    break
                fliped_frame = cv2.flip(frame, 1)

                # [Target 탐지 영역 설정 (roi 영역)]
                height, width, _ = fliped_frame.shape
                rx1, rx2 = width // 3, 2 * width // 3
                ry1, ry2 = 2 * height // 3, height - 1

                # [YOLO 추론]
                results = self.model.predict(source=fliped_frame, conf=0.25, verbose=False)
                annotated_frame = results[0].plot()
                cv2.rectangle(annotated_frame, (rx1, ry1), (rx2, ry2), (0, 0, 255), 2)
                # --- [추가: 실시간 영상 GUI 전송 로직] ---
                # 1. BGR -> RGB 변환
               # --- [이미지 전송 핵심] ---
                try:
                    rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
                    h, w, ch = rgb_image.shape
                    qt_image = QImage(rgb_image.data, w, h, ch * w, QImage.Format.Format_RGB888).copy()

                    # GUI로 데이터 발송! (이게 실행되어야 함)
                    self.image_signal.emit(qt_image)
                except Exception as e:
                    print(f"이미지 변환 에러: {e}")
                # -----------------------

                # [객체 분석]
                stop_target = False
                target_in_roi = False

                for box in results[0].boxes:
                    name = self.model.names[int(box.cls[0])]
                    b_cx, b_cy, b_width, b_height = box.xywh[0].tolist()

                    if name in stop_target_list:
                        if ((b_width * b_height) / (width * height)) >= 0.9 :
                            stop_target = True
                    elif name in signal_target_list:
                        if (rx1 <= b_cx <= rx2) and (ry1 <= b_cy <= ry2):
                            target_in_roi = True

                # --- [제어 로직] ---
                print("-" * 30)
                current_time = time.time()
                time_passed = (current_time - self.last_execution_time) > self.cooldown_duration

                if self.is_completed:
                    if target_in_roi:
                        print(">>> [위치 조정 중] 주행 종료 전 최종 위치 조정중입니다.")
                        print("move() 호출")
                        self.move(self.ADJUST)
                    else:
                        print(">>> [주행 완료] 주행을 완료하여 정지합니다.")
                        print(">>> [주행 종료] 자동 주행을 종료합니다.")
                        self.move(self.STOP)
                        cap.release()
                        self.mission_finished.emit(True)
                        return True
                elif stop_target:
                    status_text = "STOP: PERSON DETECTED"
                    display_color = (0, 0, 255)
                    # 정지하기 전, 현재가 STOP 이나 PAUSE 가 아닐 때만 상태를 백업
                    if self.current_state != self.STOP and self.current_state != self.PAUSE:
                        self.state_before_stop = self.current_state
                    print(f">>> [정지] 사람이 감지되었습니다.")
                    self.move(self.PAUSE)
                else:
                    if not target_in_roi:
                        self.target_ready_for_next = True


                    if target_in_roi and self.target_ready_for_next:
                        if time_passed:
                            print(f">>> [신호 승인] {self.all_commands[self.current_cmd_idx]} 완료. 다음 단계로!")

                            # 현재 통과한 좌표를 GUI로 전송
                            if self.drive_path and self.current_cmd_idx < len(self.drive_path):
                                curr_node = self.drive_path[self.current_cmd_idx+1]
                                print(f"({(curr_node)[0]}, {(curr_node)[1]})")
                                self.step_signal.emit(curr_node[0], curr_node[1]) # 신호 쏘기!

                            self.current_cmd_idx += 1
                            self.last_execution_time = current_time
                            self.target_ready_for_next = False

                            if self.current_cmd_idx >= total_cmds:
                                self.is_completed = True
                                status_text = "COMPLETED"
                            else:
                                status_text = "NEXT STEP LOADED"
                        else:
                            print(f">>> [위치 조정] 쿨다운 중 재인식됨. 위치조정 실행 중...")
                            print("move() 호출")
                            self.state_before_stop = self.current_state
                            self.target_ready_for_next = False
                            self.after_redetected = True
                            self.move(self.ADJUST)
                            continue

                    if not self.is_completed:
                        if not self.target_ready_for_next:
                            status_text = "WAIT: TARGET을 지나치고 있습니다."
                            print(">>> [위치 조정 중] 다음 경로 진행을 위해 위치 조정중입니다.")
                            print("move() 호출")
                            self.move(self.ADJUST)
                        else:
                            if self.is_action_finished and (not self.after_redetected):
                                # print(f">>> [수행 중] 현재 동작: {self.all_commands[self.current_cmd_idx]}")
                                current_cmd = self.all_commands[self.current_cmd_idx]
                                if "직진" in current_cmd:
                                    self.move(self.STRAIGHT)
                                elif "우회전" in current_cmd:
                                    self.move(self.TURN_RIGHT)
                                    self.move(self.STRAIGHT)
                                elif "좌회전" in current_cmd:
                                    self.move(self.TURN_LEFT)
                                    self.move(self.STRAIGHT)
                                status_text = f"STATE: {self.current_state} (MOVING)"
                                print(status_text)
                            else:
                                if self.current_state == self.PAUSE:
                                    print(f">>> {self.state_before_stop} 재개")
                                    self.move(self.state_before_stop)
                                elif self.after_redetected:
                                    print(self.state_before_stop)
                                    self.move(self.state_before_stop)
                                    self.after_redetected = False
                                else:
                                    status_text = f"STATE: {self.current_state} (IN PROGRESS...)"
                                    print(status_text)
                    display_color = (0, 255, 0)

                # [화면 표시]
                cv2.putText(annotated_frame, status_text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, display_color, 2)
                QCoreApplication.processEvents()
                time.sleep(0.01) # 0.01초만 쉬어도 클릭이 아주 잘 먹힙니다.

            cap.release()
            self.mission_finished.emit(True)
"""
---------------- Robot SW Class end -----------------
"""



"""
--------------------- GUI Class ----------------------
"""
# 로봇 얼굴 화면 (메인 1 화면)
class RobotMainScreen(QWidget):  #QWidget 상속받은 메인화면
    def __init__(self, parent=None):  # 생성자
        print("RobotMainScreen : __init__")
        super().__init__(parent)  # 부모클래스 기능 그대로
        self.setFixedSize(800, 600)  # 창 크기를 800x600으로 고정
        self.setStyleSheet("background-color: #050505;")  # 창 전체 배경색을 칠해 빈틈 방지 다크 그레이색

        # 로봇의 귀여운 뿔 (안테나)
        self.ant_l_stick = QFrame(self)  # 직선형 기둥
        self.ant_l_stick.setGeometry(270, 50, 15, 60)
        # 위젯의 위치,크기(가로,세로,너비,높이)
        self.ant_l_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        # 위젯의 모든면에서 3픽셀 두께의 흰색 선을 그려라!, 테두리 중 아래만 없애라! , 배경은 투명
        self.ant_l_tip = QFrame(self)
        self.ant_l_tip.setGeometry(252, 35, 50, 25)
        self.ant_l_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #FF8033;")  # 주황색

        self.ant_r_stick = QFrame(self)
        self.ant_r_stick.setGeometry(515, 50, 15, 60)
        self.ant_r_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        self.ant_r_tip = QFrame(self)
        self.ant_r_tip.setGeometry(497, 35, 50, 25)
        self.ant_r_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #00A3E8;")  # 하늘색, 바다색

        # 로봇 얼굴 : face_outline
        self.face_outline = QFrame(self)
        self.face_outline.setGeometry(150, 110, 500, 330)
        self.face_outline.setStyleSheet("border: 4px solid #4364F7; border-radius: 60px; background-color: #111;")
        # 로열 블루 ,모서리 끝점에서 가로 세로 60px지점을 부드러운 원형 곡선으로 ,                   #020202 보다는 살짝 더 밝은 검은색

        # 눈 버튼
        self.btn_manual = QPushButton("직접 이동", self.face_outline)
        self.btn_manual.setGeometry(60, 70, 160, 110)  # #1A1A1A 다크 그레이, #FF8033주황색, solid 는 실선, #FF8033주황색
        self.btn_manual.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #FF8033;
                font-size: 18px; font-weight: bold; border: 2px solid #FF8033; border-radius: 25px;
            }
            QPushButton:hover { background-color: #FF8033; color: black; }
        """)  # 마우스를 올릴때만 작동

        self.btn_auto = QPushButton("자동 이동", self.face_outline)
        self.btn_auto.setGeometry(280, 70, 160, 110)  # #00A3E8 맑고 깊은 파란색
        self.btn_auto.setStyleSheet("""
            QPushButton {
                background-color: #1A1A1A; color: #00A3E8;
                font-size: 18px; font-weight: bold; border: 2px solid #00A3E8; border-radius: 25px;
            }
            QPushButton:hover { background-color: #00A3E8; color: black; }
        """)

        # '=' 입 모양
        self.mouth_top = QFrame(self.face_outline)
        self.mouth_top.setGeometry(210, 240, 80, 5)
        self.mouth_top.setStyleSheet("background-color: #00FF00; border-radius: 2px;")  # #00FF00 형광연두색
        self.mouth_bottom = QFrame(self.face_outline)
        self.mouth_bottom.setGeometry(210, 255, 80, 5)
        self.mouth_bottom.setStyleSheet("background-color: #00FF00; border-radius: 2px;")

        # 하단 안내 칸
        self.bottom_bar = QFrame(self)
        self.bottom_bar.setGeometry(150, 500, 500, 70)
        self.bottom_bar.setStyleSheet("border: 2px solid white; background-color: #222; border-radius: 20px;")  #222 목탄색
        self.bottom_label = QLabel("이동 방식을 선택해주세요.", self.bottom_bar)
        self.bottom_label.setGeometry(0, 0, 500, 70)
        self.bottom_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 글자를 정중앙으로!!
        self.bottom_label.setStyleSheet("color: white; font-size: 18px; font-weight: bold; border: none;")

class ManualControlScreen(QWidget):  # 직접 이동 화면
    def __init__(self, parent=None):
        print("ManualControlScreen : __init__")
        super().__init__(parent)
        self.setFixedSize(800, 600)
        self.setStyleSheet("background-color: #0A0E14;")  # 블랙에 가까운 네이비

        # 스타일 시트
        # 이동 버튼 (Cyan네온)
        move_btn_style = """
            QPushButton {
                        /* 방사형그라데이션   ( 버튼 중앙에서 바깥으로 , 높을수록 퍼짐 , 0은 중심 약간 밝은 남색, 1은 바깥쪽 아주 어두운 남색*/
                background: qradialgradient(cx:0.5, cy:0.5, radius:0.5, stop:0 #2A3A4A, stop:1 #16212B);
                /* 2픽셀의 밝은 민트 실선   , 둥근 모서리         ,테두리와 같은 형광하늘색 */
                border: 2px solid #00F2FF; border-radius: 20px;color: #00F2FF; font-size: 35px; font-weight: bold;}
            /* 마우스를 대면            밝은 민트에서     아주 어두운 남색 */
            QPushButton:hover { background: #00F2FF; color: #0A0E14; border: 2px solid white; }
            /* 마우스를 클릭 하면    조금 더 어두운 청록색  */
            QPushButton:pressed { background: #008E9B; }
        """
        # 각도 버튼 (Orange 네온)
        #  이미 만들어 둔 디자인 틀을 그대로 쓰고         민트색 ->주황    ,  어두운 남색 -> 어두운 갈색     ,  진한 청록색  --> 진한 주황색으로
        angle_btn_style = move_btn_style.replace("#00F2FF", "#FF8033").replace("#2A3A4A", "#3D2B1F").replace("#008E9B","#CC5200")

        # 상단 제목
        self.title_frame = QFrame(self)
        self.title_frame.setGeometry(60, 40, 200, 50)     # 진한 주황
        self.title_frame.setStyleSheet("background-color: #FF8033; border-radius: 12px; border: 2px solid white;")
        self.title_label = QLabel("직접 이동", self.title_frame)
        self.title_label.setGeometry(0, 0, 200, 50)
        self.title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 정중앙
        self.title_label.setStyleSheet("color: black; font-size: 18px; font-weight: bold; border: none;")

        # 메인으로 버튼
        self.btn_back = QPushButton("메인으로", self)
        self.btn_back.setGeometry(650, 40, 110, 50)
        self.btn_back.setStyleSheet("""
            QPushButton {
                background: #1A1A1A; border: 3px solid #FF8033; border-radius: 10px;
                color: #FF8033; font-size: 16px; font-weight: bold;
            }
            QPushButton:hover { background: #FF8033; color: black; }
        """)    #  #1A1A1A 무채색 그레이

        # 위치 및 간격 설정
        move_x = 110  # 왼쪽 전체 위치  'MOVE' 섹션에 속한 모든 요소의 시작점
        angle_x = 540  # 오른쪽 전체 위치
        btn_size = 90  # 버튼 크기
        gap = 20  # 버튼 사이의 여백

        # MOVE 섹션
        self.move_tag = QLabel("▼ MOVE", self)
        self.move_tag.setGeometry(move_x, 180, 220, 30)
        self.move_tag.setStyleSheet("color: #00F2FF; font-weight: bold; font-size: 15px;")
                                           # 민트
        # 상단버튼 기준 간격 계산 배치
        self.btn_up = QPushButton("▲", self)
        self.btn_up.setGeometry(move_x + btn_size + gap, 230, btn_size, btn_size)
        self.btn_up.setStyleSheet(move_btn_style)
        self.btn_left = QPushButton("◀", self)
        self.btn_left.setGeometry(move_x, 230 + btn_size + gap, btn_size, btn_size)
        self.btn_left.setStyleSheet(move_btn_style)
        self.btn_right = QPushButton("▶", self)
        self.btn_right.setGeometry(move_x + (btn_size + gap) * 2, 230 + btn_size + gap, btn_size, btn_size)
        self.btn_right.setStyleSheet(move_btn_style)
        self.btn_down = QPushButton("▼", self)
        self.btn_down.setGeometry(move_x + btn_size + gap, 230 + (btn_size + gap) * 2, btn_size, btn_size)
        self.btn_down.setStyleSheet(move_btn_style)

        # ANGLE 섹션
        self.angle_tag = QLabel("▼ ANGLE", self)
        self.angle_tag.setGeometry(angle_x, 180, 220, 30)
        self.angle_tag.setStyleSheet("color: #FF8033; font-weight: bold; font-size: 15px;")
        # 위 버튼
        self.btn_ang_up = QPushButton("▲", self)
        self.btn_ang_up.setGeometry(angle_x + 35, 230, btn_size, btn_size);
        self.btn_ang_up.setStyleSheet(angle_btn_style)
        # 아래 버튼
        self.btn_ang_down = QPushButton("▼", self)
        self.btn_ang_down.setGeometry(angle_x + 35, 230 + (btn_size + gap) * 2, btn_size, btn_size);
        self.btn_ang_down.setStyleSheet(angle_btn_style)

class AutoMapScreen(QWidget):   # 자동 이동
    def __init__(self, parent=None):
        print("AutoMapScreen : __init__")
        super().__init__(parent)
        self.setFixedSize(800, 600)
        self.setStyleSheet("background-color: #050505;")   # 리치 블랙

        self.is_returning = False  # 처음에는 복귀모드 아님!

        # 상단 제목 (하늘색 상자)
        self.title_frame = QFrame(self)
        self.title_frame.setGeometry(30, 30, 180, 45)  # x 가 작을 수록 왼쪽
        self.title_frame.setStyleSheet("background-color: #00A3E8; border-radius: 10px; border: 2px solid white;")  # 딥스카이 블루
        self.title_label = QLabel("자동 이동", self.title_frame)
        self.title_label.setGeometry(0, 0, 180, 45)
        self.title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.title_label.setStyleSheet("color: black; font-size: 16px; font-weight: bold; border: none;")

        # "출발지를 선택해주세요." 안내 문구
        # 제목(x=30+180=210)과 메인으로 버튼(x=650) 사이 중앙에 배치
        self.guide_label = QLabel("출발지를 선택해주세요.", self)
        self.guide_label.setGeometry(210, 30, 390, 45) # x,y 너비,높이
        self.guide_label.setAlignment(Qt.AlignmentFlag.AlignCenter)                        # 1 픽셀씩 틈을 주시오!
        self.guide_label.setStyleSheet("color: #00A3E8; font-size: 18px; font-weight: bold;letter-spacing: 1px;")

        # 메인으로 버튼
        self.btn_back = QPushButton("메인으로", self)
        self.btn_back.setGeometry(650, 30, 110, 45)  #630
        self.btn_back.setStyleSheet("background: #1A1A1A; border: 2px solid #00A3E8; border-radius: 10px; color: #00A3E8; font-weight: bold;")                           # 무채색 그레이                    # 딥스카이블루

        # 버튼을 누르면 초기화 함수가 실행
        self.btn_back.clicked.connect(self.reset_screen)

        try:
            with open(my_map_path, 'r', encoding='utf-8') as f:
                my_map_data = json.load(f)
            my_map_numpy = np.array(my_map_data, dtype=object)
            is_passable = (my_map_numpy == "R") | (my_map_numpy == "C")
            self.map_data = is_passable.tolist()
            print(f">>> [Legos] Map Loaded: {my_map_path}")
        except Exception as e:
            print(f">>> [Error] Map Load Failed: {e}")

        # 전체 폭
        # offset 기준점으로부터의 거리(간격)
        self.offset_x = 40  # 작을 수록 왼쪽으로 시작점
        self.offset_y = 100  # 위쪽 여백 , 시작점
        self.cell_w = 56  # 가로 간격 커지면 옆으로
        self.cell_h = 70  # 세로 간격 커지면 아래로

        # 모든 공(버튼) 객체를 저장할 2차원 비어 있는 리스트 (7행 13열) 이게 없으면 계속 새 공으로 덮어씀.
        self.btns = [[None for _ in range(13)] for _ in range(7)]

        # 로봇 미리 준비 (처음엔 안 보이게)
        self.robot = QLabel(self)
        self.robot.setScaledContents(True)
        self.robot.hide()

        # 선택된 좌표를 저장할 공간
        self.start_pos = None  # 처음엔 비어있음 출발지 좌표 보관용
        self.end_pos = None

        self.draw_map()

    # 7 X 13
    def draw_map(self):
        print("AutoMapScreen : draw_map()")
        row = len(self.map_data)
        col = len(self.map_data[0])
        for r in range(row): # 행(세로방향) 0~6 까지 반복
            for c in range(col): # 열(가로 방향) 0~12 까지 반복
                val = self.map_data[r][c] # 현재 칸의 데이터 가져오기!
                # 실제 화면에 그려질 좌표 계산
                # 시작점(offset) + (현재 칸 번호 * 간격)
                x = self.offset_x + (c * self.cell_w)
                y = self.offset_y + (r * self.cell_h)

                # 긴 벽
                # 양 끝 열(0번째,12번째열) 이면서 데이터가 1인 경우
                if (c == 0 or c == (col-1)) and val == False:
                    if r == 0 or r == 4:   # 0행 또는 4행에서만
                        wall = QFrame(self)  # 벽 모양 틀
                        # 벽의 틈 조정
                        wall.setGeometry(x + 5, y, 25, self.cell_h * 3 - 15)
                    # x는 왼쪽에서 +5 만큼,일반셀보다 얇게 그래야 벽처럼 보이니깐,3칸을 통째로 붙이는데 아래 닿지말라고 15 빼줌
                        wall.setStyleSheet("background-color: #222; border: 1px solid #444;")
                                                    #검은색에 가까운 진한 회색    ,  중간 단계의 어두운 회색
                    continue

                if val == False:  # 네모 상자(선택불가)
                    box = QFrame(self)
                    box.setGeometry(x - 2, y -2, 45, 55)  # 네모 크기 조정 상자가 오른쪽벽에 너무 붙어서 -값 사용
                    box.setStyleSheet("background-color: #1a1a1a; border: 1px solid #555; border-radius: 5px;")
                                                       # 진한 숯색                 중간회색     0 완전 직사각형  5는 살짝 둥글게
                                                                                 # 값이 커질수록 둥글어지고 원래크기의 절반이면 원형
                elif val == True:  # 하늘색 공 (선택 가능)
                    btn = QPushButton(self)  # 클릭 버튼
                    # 위치 크기 조절
                    btn.setGeometry(x + 10, y + 12, 26, 26)
                    # 초기 하늘색 스타일 공
                    btn.setStyleSheet("""  /*  투명 배경                          하늘색    크기가 26이니깐 13이면 원형 */
                        QPushButton { background: transparent; border: 2px solid #00ffff; border-radius: 13px; }
                        QPushButton:hover { background: rgba(0, 255, 255, 0.6); }
                    """)   # 마우스를 올리면                    밝아지게      투명도
                    # 클릭시 실행될 함수 연결 (현재 행, 열 전달)
                    btn.clicked.connect(lambda ch, row=r, col=c: self.on_ball_clicked(row, col))

                    # 리스트에 버튼 저장
                    self.btns[r][c] = btn

    def on_ball_clicked(self, row, col):   # 공 클릭시 공들이 주황색으로 변하는 함수
        print("AutoMapScreen : on_ball_clicked()")
        # 출발지 선택
        if self.start_pos is None:
            self.start_pos = (row, col) # 현재 좌표를 출발지로 저장

            # 안내 문구 변경 (출발지 --> 도착지)
            self.guide_label.setText("도착지를 선택해주세요.")
            self.guide_label.setStyleSheet("color: #FF8033; font-size: 18px; font-weight: bold;")

            # 스타일 설정
            orange_border = "background: transparent; border: 2px solid #FF8033; border-radius: 13px;"
            sky_blue_fill = "background: #00A3E8; border: 2px solid white; border-radius: 13px;"

            for r in range(len(self.map_data)):
                for c in range(len(self.map_data[0])):
                    if self.btns[r][c]:  # 칸에 버튼이 있다면
                        if (r, c) == self.start_pos:
                            self.btns[r][c].setStyleSheet(sky_blue_fill) # 출발지는 파랑!
                        else:
                            self.btns[r][c].setStyleSheet(orange_border) # 나머지는 주황 테두리!
            print(f"출발지 선택 완료: {self.start_pos}")

        # 출발지가 이미 있고, 이제 도착지 선택
        else:
            if (row, col) == self.start_pos: # 출발지를 또 누르면 무시
                return
            # 클릭한 좌표를 end_pos에 저장
            self.end_pos = (row, col)

            # 도착지 강조 스타일 (진한 주황색 + 흰색 테두리)
            final_target_style = "background: #CC5200; border: 2px solid white; border-radius: 13px;"
            self.btns[row][col].setStyleSheet(final_target_style)

            # 최종 안내 문구
            self.guide_label.setText(f"목적지 확정! ({row}, {col})로 이동 시작.")
            self.guide_label.setStyleSheet("color: #CC5200; font-size: 20px; font-weight: bold;")

            # 로봇 출발 함수 실행
            self.start_robot_animation(self.start_pos, self.end_pos)

            # 2. 2초 뒤(로봇 등장 후) 실제 로봇 주행 스레드 시작

            QTimer.singleShot(4000, self.start_auto_drive)

    def start_auto_drive(self):
        print("AutoMapScreen : start_auto_drive()")

        if hasattr(self.window(), 'video_popup'):
            self.window().video_popup.show()

        # 메인 윈도우에 설정된 worker에게 일 시키기
        QCoreApplication.processEvents()
        self.window().robot_sw.request_drive.emit(self.start_pos, self.end_pos, 0)

    def update_robot_ui(self, row, col):
        # 1. 좌표 계산 (기존과 동일)
        tx = self.offset_x + (col * self.cell_w) + 10 - 7
        ty = self.offset_y + (row * self.cell_h) + 12 - 7

        # 2. 이동 및 최상단 배치
        self.robot.move(tx, ty)
        self.robot.raise_()

        # [수정] 강제로 화면을 다시 그리게 함
        self.robot.repaint() # 혹은 self.update()

    def start_robot_animation(self, start, end):
        print("AutoMapScreen : start_robot_animation()")
        # 목적지가 없으면 실행하지마라!
        if end is None: return

        if hasattr(self, 'intro_ani'):
            self.intro_ani.stop()
            self.intro_ani.deleteLater() # 기존 객체 정리

        # 출발지(start)와 목적지(end) 공만 남기고 다 숨기기
        for r in range(7):
            for c in range(13):
                if self.btns[r][c] and (r, c) != start and (r, c) != end:
                    self.btns[r][c].hide()

       # 이동 로봇
        pixmap = QPixmap("이동로봇.png")
        self.robot.setPixmap(pixmap) # 여기서 scaled를 미리 하지 말고.
        self.robot.setScaledContents(True) # 라벨 크기가 줄어들면 사진도 같이 줄어들게! 안하면 사진이 짤림.

        # png 투명도 살리는 핵심 설정
        self.robot.setAttribute(Qt.WidgetAttribute.WA_TranslucentBackground)
        self.robot.setStyleSheet("background:transparent; border:none;") # 잔여 배경 제거

        # 처음 등장할 때 위치와 크기 (200x200)
        self.robot.setGeometry(300, 50, 200, 200)   # 시작위치 중앙위쪽
        self.robot.show() # 이제 보여줌
        self.robot.raise_()  # 로봇이 다른 그림들 보다 위에 오개!

        # 출발지 실제 좌표 계산 (작아졌을 때 갈 곳)
        # 공의 크기가 26x26이므로, 로봇(40x40)을 공 정중앙에 올리기 위한 보정값(-7)
        start_x = self.offset_x + (start[1] * self.cell_w) + 10-7
        start_y = self.offset_y + (start[0] * self.cell_h) + 12-7

        # 등장 애니메이션: 중앙(크게) -> 출발지(작게)
        self.intro_ani = QPropertyAnimation(self.robot, b"geometry")
        self.intro_ani.setDuration(3000) # 2초 동안 실행 조금 더 부드럽게
        self.intro_ani.setStartValue(QRect(300, 50, 200, 200)) # 시작 위치/크기
        self.intro_ani.setEndValue(QRect(start_x, start_y, 40, 40)) # 출발지 위치
        self.intro_ani.setEasingCurve(QEasingCurve.Type.OutQuart) # 실선 방지용 부드러운 곡선

        self.intro_ani.finished.connect(lambda: self.robot.setGeometry(start_x, start_y, 40, 40))
        self.intro_ani.start()
        # 안내 문구 변경
        self.guide_label.setText("로봇이 준비 중입니다...")

    def on_mission_finished(self, success):
            print("AutoMapScreen : on_mission_finished()")
            if success:
                self.guide_label.setText("목적지에 도착했습니다! 주행 완료.")
                self.guide_label.setStyleSheet("color: #00FF00; font-size: 20px; font-weight: bold;")

                if hasattr(self.window(), 'video_popup'):
                    self.window().video_popup.close()

                # 2. [세리머니] 로봇 점프 애니메이션 시작
                curr_rect = self.robot.geometry()
                self.jump_ani = QPropertyAnimation(self.robot, b"geometry")
                self.jump_ani.setDuration(500)
                self.jump_ani.setStartValue(curr_rect)
                self.jump_ani.setKeyValueAt(0.5, QRect(curr_rect.x(), curr_rect.y() - 30, 40, 40))
                self.jump_ani.setEndValue(curr_rect)
                self.jump_ani.setLoopCount(3) # 3번 점프
                self.jump_ani.setEasingCurve(QEasingCurve.Type.OutCubic)

                # 3. 점프가 다 끝나면 다이얼로그를 띄우도록 연결
                self.jump_ani.finished.connect(self.check_where_we_are)
                self.jump_ani.start()

    def check_where_we_are(self):
        print("AutoMapScreen : check_where_we_are()")
        # 만약 목적지 좌표와 출발지 좌표가 '복귀' 상태로 뒤집히면
        # 즉 로봇이 현재 (0,0) 근처인 초기 출발지에 와 있다면 복귀 완료창!
        if hasattr(self, 'is_returning') and self.is_returning:
            dlg = ReturnFinishDialog(self)
            if dlg.exec():
                self.is_returning = False # 모드 초기화
                self.reset_screen()       # 지도 초기화
                self.window().stack.setCurrentIndex(0) # 메인 얼굴 화면으로!
        else:
            # 일반 도착 시
            dlg = MyFinishDialog(self)
            if dlg.exec():
                self.go_to_next_screen()

    def go_to_next_screen(self): # 화면전환
        print("AutoMapScreen : go_to_next_screen()")
        # 이 화면이 속한 전체 창(LegosApp)을 찾음.
        boss = self.window()
        # 대장(LegosApp)에게 "index 3번으로 바꾸라고 명령.
        if hasattr(boss, 'stack'): # has attribute 약자 , index 3번 있소?
            boss.stack.setCurrentIndex(3)

    def reverse_path_and_move(self): # 복귀시작
        print("AutoMapScreen : reverse_path_and_move()")
        self.is_returning = True # "나 지금 복귀 중이야"라고 표시!
        # 기존 출발지와 목적지 바꾸기!
        temp = self.start_pos
        self.start_pos = self.end_pos
        self.end_pos = temp

        sky_blue_fill = "background: #00A3E8; border: 2px solid white; border-radius: 13px;"
        final_target_style = "background: #CC5200; border: 2px solid white; border-radius: 13px;"
        self.btns[self.start_pos[0]][self.start_pos[1]].setStyleSheet(sky_blue_fill)
        self.btns[self.end_pos[0]][self.end_pos[1]].setStyleSheet(final_target_style)

        # 현재 화면을 잠시 유지하며 안내 문구만 변경
        self.guide_label.setText("잠시 후 출발지로 복귀를 시작합니다...")
        self.guide_label.setStyleSheet("color: #FF8033; font-size: 20px; font-weight: bold;")

        # 1. 로봇 아이콘을 출발지에 세팅 (애니메이션 포함)
        self.start_robot_animation(self.start_pos, self.end_pos)

        # 2. 2초 뒤(로봇 등장 후) 실제 로봇 주행 스레드 시작
        QTimer.singleShot(4000, self.start_auto_drive)

    def reset_screen(self):
        print("AutoMapScreen : reset_screen()")
        # 저장된 좌표 데이터 초기화
        self.start_pos = None
        # 로봇도 숨기기
        self.robot.hide()

        # 안내 문구와 제목 스타일을 처음 하늘색 상태로!
        self.guide_label.setText("출발지를 선택해주세요.")
        self.guide_label.setStyleSheet("color: #00A3E8; font-size: 18px; font-weight: bold; letter-spacing: 1px;")
        #self.title_frame.setStyleSheet("background-color: #00A3E8; border-radius: 10px; border: 2px solid white;")

        # 모든 공의 색깔을 다시 원래(기본값)의 투명한 하늘색 테두리로!
        default_style = "background: transparent; border: 2px solid #00ffff; border-radius: 13px;"
        for r in range(7):
            for c in range(13):
                if self.btns[r][c]: # 버튼이 있는 칸만 골라서
                    self.btns[r][c].show() # ★ 숨겼던 공들을 다시 보여주기!
                    self.btns[r][c].setStyleSheet(default_style)

class MyFinishDialog(QDialog):  # QDialog 대화창의 모든 기능
    def __init__(self, parent=None):
        print("MyFinishDialog : __init__")
        super().__init__(parent)
        # 기존의 플래그들을 유지하면서 상단바만 없애는 방식 (| 연산자 사용) | 안쓰면 앞에 인트로 문열리는거 사라짐.
        self.setWindowFlags(self.windowFlags() | Qt.WindowType.FramelessWindowHint)
        self.setFixedSize(450, 220) # 창 크기
        # 맵 배경과 어울리는 어두운 배경색          다크그레이                    하늘색
        self.setStyleSheet("background-color: #2b2b2b; border: 2px solid #00AAFF; border-radius: 12px;")

        layout = QVBoxLayout() # 레이아웃 박스
        layout.setContentsMargins(20, 30, 20, 30) # 여백 조절 (왼,위,오,아래)

        # 메인 글자 설정                                                                              바깥쪽 여백   -는 아래로
        self.label = QLabel('목적지에 무사히 도착! <img src="이동로봇.png" width="45" height="45" style="margin-bottom: -15px;>')
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        # 글자 크기를 키우고 하얗게 설정
        self.label.setStyleSheet("color: white; font-size: 32px; font-weight: bold; border: none;")

        # 치우친 그림자 효과 적용
        shadow = QGraphicsDropShadowEffect()
        shadow.setBlurRadius(0)          # 그림자를 뭉개지 않고 선명하게
        shadow.setOffset(5, 5)           # 오른쪽 아래로 5px 치우치게! x축은 + 오른쪽 , y 축은 + 아래쪽
        shadow.setColor(QColor(0, 0, 0)) # 완전 검은색 그림자
        self.label.setGraphicsEffect(shadow)

        # 확인 버튼 설정
        self.ok_btn = QPushButton("확인")
        self.ok_btn.setObjectName("final_ok_button")
        self.ok_btn.setFixedSize(140, 50)
        self.ok_btn.setStyleSheet("""
            QPushButton#final_ok_button {
                background-color: #00A3E8 !important;
                color: #ffffff !important;
                border: 2px solid white !important;
                font-size: 20px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #0081B9;
            }
        """)   ##00A3E8 맑은 파란색  #0081B9 좀더 진한 색 !important 붙이면 무조건 흰색
        # 확인 버튼 누르면 창 닫기
        self.ok_btn.clicked.connect(self.accept)

        # 레이아웃에 배치
        layout.addWidget(self.label) # 글자 맨위
        layout.addSpacing(20) # 글자와 버튼 사이 간격 20 비우공
        layout.addWidget(self.ok_btn, alignment=Qt.AlignmentFlag.AlignCenter) # 확인버튼은 센터로!
        self.setLayout(layout) # 보여줘!

class ReturnFinishDialog(QDialog):
    def __init__(self, parent=None):
        print("ReturnFinishDialog : __init__")
        super().__init__(parent)
        self.setWindowFlags(self.windowFlags() | Qt.WindowType.FramelessWindowHint)
        self.setFixedSize(450, 220)
        # 복귀 완료는 초록색(#28A745) 테두리로 포인트!
        self.setStyleSheet("background-color: #1a1a1a; border: 2px solid #28A745; border-radius: 12px;")

        layout = QVBoxLayout()
        # 로봇이동과 함께 문구 출력
        self.label = QLabel('출발지로 복귀 완료! <img src="이동로봇.png" width="45" height="45">')
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setStyleSheet("color: white; font-size: 28px; font-weight: bold; border: none;")

        self.ok_btn = QPushButton("확인(메인으로)")
        self.ok_btn.setFixedSize(200, 50)
        self.ok_btn.setStyleSheet("""
            QPushButton {
                background-color: #28A745; color: white;
                border: 2px solid white; font-size: 18px; font-weight: bold; border-radius: 10px;
            }
            QPushButton:hover { background-color: #218838; }
        """)  # #28A745 초록색 , #218838 앞에 초록 보단 조금 더 진한 초록
        self.ok_btn.clicked.connect(self.accept)

        layout.addWidget(self.label)
        layout.addSpacing(20)
        layout.addWidget(self.ok_btn, alignment=Qt.AlignmentFlag.AlignCenter)
        self.setLayout(layout)

class MissionCompleteScreen(QWidget):
    def __init__(self, parent=None):
        print("MissionCompleteScreen : __init__")
        super().__init__(parent)
        self.setFixedSize(800, 600)  # 창 크기를 800x600으로 고정
        self.setStyleSheet("background-color: #050505;") # 배경은 검은색!

        # 로봇의 귀여운 뿔 (안테나)
        self.ant_l_stick = QFrame(self)  # 직선형 기둥
        self.ant_l_stick.setGeometry(270, 50, 15, 60)
        # 위젯의 위치,크기(가로,세로,너비,높이)
        self.ant_l_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        # 위젯의 모든면에서 3픽셀 두께의 흰색 선을 그려라!, 테두리 중 아래만 없애라! , 배경은 투명
        self.ant_l_tip = QFrame(self)
        self.ant_l_tip.setGeometry(252, 35, 50, 25)
        self.ant_l_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #FF8033;")  # 주황색

        self.ant_r_stick = QFrame(self)
        self.ant_r_stick.setGeometry(515, 50, 15, 60)
        self.ant_r_stick.setStyleSheet("border: 3px solid white; border-bottom: none; background: transparent;")
        self.ant_r_tip = QFrame(self)
        self.ant_r_tip.setGeometry(497, 35, 50, 25)
        self.ant_r_tip.setStyleSheet("border: 3px solid white; border-radius: 12px; background: #00A3E8;")  # 하늘색, 바다색

        # 로봇 얼굴 : face_outline
        self.face_outline = QFrame(self)
        self.face_outline.setGeometry(150, 110, 500, 330)
        self.face_outline.setStyleSheet("border: 4px solid #28A745; border-radius: 60px; background-color: #111;")
        # 성공 초록색 ,모서리 끝점에서 가로 세로 60px지점을 부드러운 원형 곡선으로 , #020202 보다는 살짝 더 밝은 검은색

        # 출발지로 복귀 버튼
        self.btn_return = QPushButton("출발지로 복귀", self.face_outline)
        self.btn_return.setGeometry(60, 70, 160, 110)  # #28A745 성공 초록, #218838 진초록
        self.btn_return.setStyleSheet("""
            QPushButton {
                background-color: #28A745; color: white;
                font-size: 18px; font-weight: bold;border-radius: 25px;
            }
            QPushButton:hover { background-color: #218838; }
        """)  # 마우스를 올릴때만 작동
        # 메인으로 버튼
        self.btn_reset = QPushButton("메인으로", self.face_outline)
        self.btn_reset.setGeometry(280, 70, 160, 110) # #FFC107 황금색, #E0A800 다크 노랑
        self.btn_reset.setStyleSheet("""
            QPushButton {
                background-color: #FFC107; color: black;
                font-size: 18px; font-weight: bold;border-radius: 25px;
            }
            QPushButton:hover { background-color: #E0A800; }
        """)

        # 하단 안내 문구 추가 (검정 바 안에 흰색 글씨)
        self.info_bar = QFrame(self)
        self.info_bar.setGeometry(150, 480, 500, 60) # 짙은 차콜라                       앞에보다 더 짙은 차콜라
        self.info_bar.setStyleSheet("background-color: #222; border-radius: 30px; border: 2px solid #555;")

        self.info_label = QLabel("이동을 완료하였습니다.", self.info_bar)
        self.info_label.setGeometry(0, 0, 500, 60)
        self.info_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.info_label.setStyleSheet("color: white; font-size: 22px; font-weight: bold; background: transparent;")

        # '=' 입 모양
        self.mouth_top = QFrame(self.face_outline)
        self.mouth_top.setGeometry(210, 240, 80, 5)
        self.mouth_top.setStyleSheet("background-color: #00FF00; border-radius: 2px;")  # #00FF00 형광연두색
        self.mouth_bottom = QFrame(self.face_outline)
        self.mouth_bottom.setGeometry(210, 255, 80, 5)
        self.mouth_bottom.setStyleSheet("background-color: #00FF00; border-radius: 2px;")

class VideoWindow(QWidget):
    emergency_signal = pyqtSignal()

    def __init__(self):
        print("VideoWindow : __init__")
        super().__init__()
        self.setWindowTitle("Robot Camera Feed")
        self.resize(640, 600) # setFixedSize 대신 resize 권장
        self.setStyleSheet("background-color: black;")




        # 레이아웃을 써야 라벨이 창 크기에 맞춰 자동으로 조절됩니다.
        self.layout = QVBoxLayout(self)
        self.layout.setContentsMargins(0, 0, 0, 0)
        self.layout.setContentsMargins(15, 15, 15, 15)
        self.layout.setSpacing(15)

        self.video_label = QLabel("영상 대기 중...")
        self.video_label.setScaledContents(True)
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.video_label.setStyleSheet("color: white;")
        self.layout.addWidget(self.video_label, stretch=1)

        self.btn_stop = QPushButton("EMERGENCY STOP")
        self.btn_stop.setMinimumHeight(50)
        self.btn_stop.setStyleSheet("""
            background-color: #FF0000;
            color: white;
            font-weight: bold;
            font-size: 20px;
            border-radius: 5px;
        """)
        self.btn_stop.clicked.connect(self.emergency_signal.emit) # 클릭 시 시그널 발송
        self.layout.addWidget(self.btn_stop, stretch=0)

    def update_frame(self, qt_image):

        if qt_image.isNull(): return

        # 이미지를 현재 라벨 크기에 딱 맞게 조절
        pixmap = QPixmap.fromImage(qt_image)
        self.video_label.setPixmap(pixmap)
        self.video_label.repaint() # 강제로 화면 다시 그리기 호출
        print("VideoWindow : update_frame")

    def clear_frame(self):
        """이전 영상을 지우고 초기 상태로 복구"""
        self.video_label.clear()  # 설정된 pixmap 제거
        self.video_label.setText("영상 대기 중...")
        # 레이아웃과 배경색을 유지하며 즉시 갱신
        self.video_label.repaint()
        print("VideoWindow : Frame Cleared")


# 대장 클래스 (전체 배경색 적용)
class LegosApp(QWidget):  # 전체 GUI 담당
    def __init__(self):
        super().__init__()
        self.setFixedSize(800, 600)
        self.setWindowFlags(Qt.WindowType.FramelessWindowHint)  # 위쪽의 닫기/최소화바 없앰 (가로,세로 테두리 없는 창)
        self.setStyleSheet("background-color: #050505;")  # 전체 앱 배경을 검은색으로 고정하여 테두리 빈틈 방지

        # 스레드 설정
        self.worker_thread = QThread()
        self.robot_sw = LegosRobot(my_model_path) # 방금 수정한 클래스
        self.robot_sw.load_map(my_map_path)
        self.robot_sw.moveToThread(self.worker_thread)
        self.worker_thread.start()

        # 슬라이드 박스 생성
        self.layout = QVBoxLayout(self)
        self.layout.setContentsMargins(0, 0, 0, 0)
        self.stack = QStackedWidget()  # 화면 쌓기
        self.layout.addWidget(self.stack)

        # 화면 등록
        self.main_view = RobotMainScreen()  # Index 0: 로봇 얼굴
        self.stack.addWidget(self.main_view)

        self.manual_view = ManualControlScreen()  # Index 1: 직접 이동
        self.stack.addWidget(self.manual_view)

        self.map_view = AutoMapScreen()  # Index 2: 자동 이동
        self.stack.addWidget(self.map_view)

        self.complete_view = MissionCompleteScreen()  # Index 3: 이동완료
        self.stack.addWidget(self.complete_view)

        self.video_popup = VideoWindow()


        # 3. 신호 연결 (가장 중요!)
        # 로봇 SW(Signal) -> GUI(Slot)
        self.robot_sw.step_signal.connect(self.map_view.update_robot_ui)
        self.robot_sw.mission_finished.connect(self.map_view.on_mission_finished)
        self.robot_sw.mission_finished.connect(self.video_popup.clear_frame)
        self.robot_sw.status_signal.connect(self.map_view.guide_label.setText)
        self.robot_sw.image_signal.connect(self.video_popup.update_frame)
        self.robot_sw.status_signal.connect(self.safe_label_update)
        self.video_popup.emergency_signal.connect(self.robot_sw.stop_immediately)
        self.video_popup.emergency_signal.connect(self.video_popup.clear_frame)
        self.video_popup.emergency_signal.connect(self.video_popup.close)
        self.video_popup.emergency_signal.connect(
            lambda: self.safe_label_update("🚨 비상 정지됨!")
        )

        # 버튼과 인덱스 번호 맞추기
        # 직접 이동 버튼 -> Index 1번 화면으로
        self.main_view.btn_manual.clicked.connect(lambda: self.stack.setCurrentIndex(1))
        # 클릭시 stack 에서 1번화면을 꺼내줘!   # lambda 사용이용 : 클릭하는 순간 실행되도록 하기 위해!

        # 자동 이동 버튼 -> Index 2번 화면으로
        self.main_view.btn_auto.clicked.connect(lambda: self.stack.setCurrentIndex(2))

        def start_auto_mode():
            self.stack.setCurrentIndex(2)  # 지도 화면으로 이동
            # self.video_popup.show()        # 영상 팝업 창 띄우기

        self.main_view.btn_auto.clicked.connect(start_auto_mode)

        # 각 화면의 '메인으로' 버튼 -> Index 0번(얼굴)으로
        self.manual_view.btn_back.clicked.connect(lambda: self.stack.setCurrentIndex(0))
        self.map_view.btn_back.clicked.connect(lambda: self.stack.setCurrentIndex(0))

        # [메인으로] 누르면 메인(0번)으로 가며 리셋
        self.complete_view.btn_reset.clicked.connect(self.go_to_main_and_reset)
        # [출발지로 복귀]는 출발지로 돌아가는 화면!
        self.complete_view.btn_return.clicked.connect(self.start_return_trip)

        self.setup_intro()

    def safe_label_update(self, text):
        self.map_view.guide_label.setText(text)
        # processEvents 대신 특정 위젯만 즉시 갱신
        self.map_view.guide_label.repaint()

    def start_return_trip(self):  # 출발지로 복귀
        self.stack.setCurrentIndex(2) # 지도 화면으로 이동
        self.video_popup.show()        # 영상 팝업 창 띄우기
        if hasattr(self.map_view, 'reverse_path_and_move'):
            self.map_view.reverse_path_and_move() # 로봇 좌표 뒤집기

    def go_to_main_and_reset(self):  # 메인으로 리셋과 함께
        # 일단 메인 화면(0번)으로 이동
        self.stack.setCurrentIndex(0)

        # 지도 화면(map_view)이 가지고 있는 초기화 함수를 호출!
        if hasattr(self.map_view, 'reset_screen'):
            self.map_view.reset_screen()

    def setup_intro(self):
        # 인트로 레이어 (맨 마지막!)
        self.intro_layer = QFrame(self)
        # 인트로 레이어 생성 (항상 맨 마지막에 만들어야 모든 화면을 덮는다! 딴거보다 먼저 만들면 엉뚱한 화면이 덮을 수도 있다.)
        self.intro_layer.setGeometry(0, 0, 800, 600)
        self.intro_layer.setStyleSheet("background-color: #050505;")  # 완전 검은색
        self.intro_layer.show()   # 화면에 보이기
        self.intro_layer.raise_()  # 지도보다 위로 올리기

        # 로고 설정 (인트로 레이어 소속)
        self.logo = QLabel("LEGOS", self.intro_layer)
        self.logo.setAlignment(Qt.AlignmentFlag.AlignCenter)  # 정중앙
        self.logo.setGeometry(0, 225, 800, 150)  # 화면 중앙 높이쯤 배치
        self.current_spacing = -60  # 처음에 글자 뭉치게 자간 -60
        self.update_logo_style()  # 아래의 정의된 스타일 함수 호출
        # 로고가 '스르륵' 나타나게 해주는 투명도 필름(Effect) 장착
        self.logo_op = QGraphicsOpacityEffect(self.logo)
        self.logo.setGraphicsEffect(self.logo_op)
        self.logo_op.setOpacity(0)  # 시작은 투명 0 (안보임)
        # 문 2개 설정 (인트로 레이어 소속)
        self.left_door = QFrame(self.intro_layer)
        self.right_door = QFrame(self.intro_layer)
        self.setup_full_design_doors()  # 디자인 (못,손잡이, 경고띠) 입힘
        # 문 흔들기 애니메이션 시작 진동을 제어할 변수와 타이머 준비
        self.shake_count = 0  # 몇번 진동 했는지 기록
        self.shake_timer = QTimer(self)  # 째깍거리는 시계
        self.shake_timer.timeout.connect(self.do_shake)  # 시계가 울릴때마다 do_shake 실행 무작위로 x,y 이동! 진동 효과
        QTimer.singleShot(1000, self.start_shaking)  # 1초 후에 start_shaking 실행

    def update_logo_style(self):  # 로고 스타일
        self.logo.setStyleSheet(
            #글자 색상 : 선형그래디언트   시작점(0,0)위, 끝점(0,1)아래,위쪽끝:연한파랑,중간색 : 진한 파랑, 아래끝:짙은 파랑 ,   크기   :   150
            f"color: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #6FB1FC, stop:0.5 #4364F7, stop:1 #0052D4); font-size: 150px; font-weight: bold; letter-spacing: {self.current_spacing}px; background: transparent;")
                      # 굵게 ,   글자 간격       -60~15  널게 벌어짐       , 글자배경 :  투명 배경없음,배경색 그대로

    def setup_full_design_doors(self):  # 문 디테일
        # 오리지널 CSS 스타일 그대로 적용 CSS 코드들 변수에 저장(가독성위해)
        door_base_css = "background: #111; border: 1px solid #222;"  # 문 바디
                  # 배경 : 진한 검정색  , 테두리 1픽셀 두께 , 어두운 회색
        #패널 문안쪽 장식판 # 배경                   왼쪽   위   , 오른쪽 아래   대각선 방향으로 밝은회색->짙은회색     테두리 : 2픽셀, 회색
        panel_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #2a2a2a, stop:1 #151515); border: 2px solid #3d3d3d; border-radius: 4px;"  # 모서리 살짝 둥글게
        # 못(리벳)    # 배경       원형그라디언트       중심         , 반지름       중심색:연회색, 바깥색 : 검정  , 동그란 못 모양
        rivet_css = "background: qradialgradient(cx:0.4, cy:0.4, radius:0.7, stop:0 #ccc, stop:1 #111); border-radius: 7px;"
        # 손잡이  (메탈 느낌)                        왼쪽       , 오른쪽      , 왼쪽끝  검정   중앙     흰색   오른쪽끝 검정  ,  어두운 테두리
        handle_css = "background: qlineargradient(x1:0, y1:0, x2:1, y2:0, stop:0 #000, stop:0.5 #fff, stop:1 #000); border: 1px solid #444;"
        # 경고띠                      선형 그라디언트     반복       ,           반복되는 선 비율조정           노랑
        hazard_css = "background: qlineargradient(spread:reflect, x1:0, y1:0, x2:0.4, y2:0.1, stop:0 #fbc02d, stop:0.5 #fbc02d, stop:0.51 #000, stop:1 #000);"  # 검정 교차

        for door, side in [(self.left_door, "L"), (self.right_door, "R")]:  # L,R 구분위해 side 사용
            # 위젯 위치와 크기 지정                          너비,높이 (문크기)
            door.setGeometry(0 if side == "L" else 400, 0, 400, 600)
                          # 왼쪽(0,0,400,600) , 오른쪽 (400,0,400,600)
            door.setStyleSheet(door_base_css)  # "background: #111; 기본값(border: 1px solid #222;"검정, 1픽셀의 짙은회색)

            # 1. 직사각형 패널
            panel = QFrame(door)  # 사각형 빈틀
            panel.setGeometry(80, 100, 240, 400)
            # 위치 왼쪽,오른쪽 숫자만큼 떨어진,크기
            panel.setStyleSheet(panel_css)

            # 2. 못 (리벳)
            pts = [(15, 15), (210, 15), (15, 370), (210, 370)]  # 못 위치
            for px, py in pts:
                rv = QFrame(panel)
                rv.setGeometry(px, py, 14, 14)  # 크기
                rv.setStyleSheet(rivet_css)

            # 3. 메탈 손잡이
            h = QFrame(door)
            h.setGeometry(385 if side == "L" else 0, 250, 15, 100)
            # 위치  :문 오른쪽 끝       ,  문 왼쪽 끝 근처
            h.setStyleSheet(handle_css)

            # 4. 공장 무늬 (경고띠)
            haz = QFrame(door)
            haz.setGeometry(0, 560, 400, 40)
                    # 위치  문 하단  , 너비 , 높이
            haz.setStyleSheet(hazard_css)

    def start_shaking(self):
        self.shake_timer.start(15)  # 0.015 초마다 진동 알람.

    def do_shake(self):  # 알람이 울릴 때마다 실행되는 함수
        power = 10 + (self.shake_count // 20)
        # 강도 기본 10  + 20번 마다 1px 증가
        dx, dy = random.randint(-power, power), random.randint(-power, power)  # 랜덤 가로, 세로 값
        self.left_door.move(dx, dy)
        self.right_door.move(400 + dx, dy)  # 문을 랜덤위치로~
        self.shake_count += 1
        if self.shake_count > 80:  # 80번까지 진동!
            self.shake_timer.stop()  # 알람 stop
            self.left_door.move(0, 0)
            self.right_door.move(400, 0)  # 문 위치 다시 0으로 정렬
            self.start_opening()  # 문열기

    def start_opening(self):
        self.group = QParallelAnimationGroup(self)  # 동시에 여러 애니메이션 돌리기 위한 grop
        # 2.4초 동안 문을 화면 밖으로 완전히 밀어냄
        # QPropertyAnimation PyQt에서 위젯 속성을 시간에 따라 애니메이션 하는 클래스
        # PyQt의 QPropertyAnimation은 속성 이름을 문자열이 아닌 바이트 문자열로 받도록 되어 있음 그래서 b. 이렇게 씀
        # geometry = QRect(x, y, width, height)
        # 왼쪽문 애니메이션
        self.a1 = QPropertyAnimation(self.left_door, b"geometry")
                 # 애니메이션 적용 위젯, 속성(위치, 크기)
        self.a1.setDuration(2400) # 2.4초 움직임
        self.a1.setStartValue(QRect(0, 0, 400, 600)) # 시작 위치 추가!
        self.a1.setEndValue(QRect(-400, 0, 400, 600))  # 완쪽 화면 밖 , 문크기 유지
        # 오른쪽 문
        self.a2 = QPropertyAnimation(self.right_door, b"geometry")
        self.a2.setDuration(2400)
        self.a2.setStartValue(QRect(400, 0, 400, 600)) # 시작 위치 추가!
        self.a2.setEndValue(QRect(800, 0, 400, 600))
        # 로고 투명도
        self.a3 = QPropertyAnimation(self.logo_op, b"opacity")
        # QLabel로고에 적용한 투명도 효과,속성
        self.a3.setDuration(2500)
        self.a3.setStartValue(0) # 시작 투명도 0 추가!
        self.a3.setEndValue(1)  # 투명도 1 완전히 보임 문이 열리는 즉시 로고 보임.

        self.group.addAnimation(self.a1)
        self.group.addAnimation(self.a2)
        self.group.addAnimation(self.a3)
        self.anim_group = self.group
        self.group.start()  # 3개의 애니메이션 동시 시작!

        # 글자 자간을 벌리는 타이머도 별도로 작동
        self.st = QTimer(self)
        self.st.timeout.connect(self.anim_text)
        self.st.start(30)

    def anim_text(self):
        if self.current_spacing < 15:  # 자간이 15 될 때까지
            self.current_spacing += 1.0
            self.update_logo_style()  # 1픽셀씩 벌리고 로고 스타일 업데이트
        else:
            self.st.stop()  # 다 벌어지면 stop
            # 연출 감상을 위해 1.8초 대기 후 인트로 종료 함수 실행
            QTimer.singleShot(1800, self.finish_intro)

    def finish_intro(self):
        # 실행 되는 순간 인트로 레이어를 삭제하며 밑에 있던 메인 화면이 매끄럽게 보임
        # self.intro_layer.deleteLater() 삭제 대신 숨기기 자꾸 문 스르륵이 없어짐!
        self.intro_layer.hide()

    def keyPressEvent(self, event):
        print("LegosApp : keyPressEvent()")
        if event.key() == Qt.Key.Key_Escape: self.close()  # esc 누르면 꺼짐.
"""
------------------- GUI Class end --------------------
"""



"""
----------------------- Main() ------------------------
"""
# 실행
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = LegosApp()
    window.show()
    sys.exit(app.exec())
"""
--------------------- Main() end ----------------------
"""

WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
>>> [Legos] Map Loaded: map_data.json
RobotMainScreen : __init__
ManualControlScreen : __init__
AutoMapScreen : __init__
>>> [Legos] Map Loaded: map_data.json
AutoMapScreen : draw_map()
MissionCompleteScreen : __init__
VideoWindow : __init__
AutoMapScreen : on_ball_clicked()
출발지 선택 완료: (0, 3)
AutoMapScreen : on_ball_clicked()
AutoMapScreen : start_robot_animation()
AutoMapScreen : start_auto_drive()
run_auto_drive
Loading best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 with CPUExecutionProvider
VideoWindow : update_frame
------------------------------
>>> [신호 승인] 1단계: 직진 (Go Straight) 완료. 다음 단계로!
(1, 3)
>>> [위치 조정 중] 다음 경로 진행을 위해 위치 조정중입니다.
move() 호출
>>> [MOVE] 신호 발생: 7 (위치조정)
VideoWindow : update_frame
------------------------------
>>> [위치 조정 중] 다음 경로 진행을 위해 위치 조정중입니다.
move() 호출
>>> [MOVE] 신호 발생: 

SystemExit: 0

C:\Code\PythonProject\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
